In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
print(scipy.__version__)

from pympler import asizeof
import gpflow
import tensorflow_probability as tfp

from matplotlib import pyplot as plt

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 16:46:05.477218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 16:46:05.477277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 16:46:05.478985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 16:46:05.487547: I tensorflow/core/platform/cpu_feature_guard

1.13.1


In [2]:
#Set Parameters
cs_name_val = 1
ep0 = 1 #Set initial ep as an even mix between exploration and exploitation
ep_enum_val = 1
meth_name_val = 1
sep_fact = 1.0
gen_heat_map_data = False
normalize = True
noise_mean = 0
noise_std = None
kernel_enum_val = 1
lenscl = None #list([0.136113749, 221.573761, 830.968019, 1.67917241, 0.3, 0.2])
outputscl = None
retrain_GP = 5
reoptimize_obj = 5
bo_iter_tot = 5
bo_run_total = 2
runs_per_job_max = 2
bo_runs_in_job = bo_run_total
save_data = False
ei_tol = 1e-7
obj_tol = 1e-7
num_x_data = 5
gen_meth_theta = 1
gen_meth_x = 2
gen_meth_theta_val = 1
num_val_pts = 0
num_theta_multiplier = 10 #How many simulation data points to generate is equal to num_theta_multiplier*number of parameters
seed = 1
sim_seed = 1
noise_std_pct = 0.01

In [3]:
# Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(meth_name_val)
method = GPBO_Methods(meth_name)
ep_enum = Ep_enum(ep_enum_val)
kernel = Kernel_enum(kernel_enum_val)
lenscl = lenscl
try:
    lenscl = json.loads(lenscl)
except:
    lenscl = lenscl


#All simulator objects will have the same seed. This keeps restarts/jobs consistent for data generation
simulator = simulator_helper_test_fxns(
    cs_name_val, noise_mean, noise_std, sim_seed
)

# Generate Exp Data (OR Add your own experimental data as a Data class object)
gen_meth_x = Gen_meth_enum(gen_meth_x)
if cs_name_val == 16:
    x_vals = np.array([0.0,0.1115,0.2475,0.4076,0.5939,0.8230,0.9214,0.9296,0.985,1.000])
elif cs_name_val == 17:
    x_vals = np.array([0.0087,0.0269,0.0568,0.1556,0.2749,0.4449,0.661,0.8096,0.9309,0.9578])    
else:
    x_vals = None

# Set simulator noise_std artifically as 1% of y_exp median (So that noise will be set rather than trained)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, x_vals, noise_std_pct)
simulator.noise_std = np.abs(np.median(exp_data.y_vals)) * noise_std_pct #Manually set noise std

# Create Exploration Bias Class
if ep_enum.value == 1:
    # Constant value stays constant
    ep_bias = Exploration_Bias(
        ep0, None, ep_enum, None, None, None, None, None, None, None
    )
elif ep_enum.value == 2:
    # For decay method, decay from mixed to full exploitation (alpha of 0.5) for this example
    ep_bias = Exploration_Bias(
        ep0,
        None,
        ep_enum,
        None,
        bo_iter_tot,
        None,
        0.5,
        None,
        None,
        None,
    )
elif ep_enum.value == 3:
    # Set ep multiplier to 1.5 as recommended in Boyle
    ep_bias = Exploration_Bias(
        ep0, None, ep_enum, None, None, 1.5, None, None, None, None
    )
elif ep_enum.value == 4:
    # Jasrasaria method will take care of itself
    ep_bias = Exploration_Bias(
        None, None, ep_enum, None, None, None, None, None, None, None
    )
else:
    raise Warning("Ep_enum value must be between 1 and 4!")

# Generate Sim (Training) Data (OR Add your own training data here as a Data class object)
num_theta_data = len(simulator.indices_to_consider) * num_theta_multiplier
gen_meth_theta = Gen_meth_enum(gen_meth_theta)
# Note at present, training data is always the same between jobs since we set the data generation seed to 1    
sim_data = simulator.gen_sim_data(
    num_theta_data,
    num_x_data,
    gen_meth_theta,
    gen_meth_x,
    sep_fact,
    sim_seed,
    False,
    x_vals
)

y_to_sse = False
# Gen sse_sim_data and sse_sim_val_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(
    method, sim_data, exp_data, sep_fact, y_to_sse
)

# Generate validation data if applicable. This is only useful for small (<4 Params + 1 State Point). Otherwise this takes up too much memory
if num_val_pts > 0:
    gen_meth_theta_val = Gen_meth_enum(
        gen_meth_theta_val
    )  # input is an integer (1 or 2)
    val_data = simulator.gen_sim_data(
        num_val_pts,
        num_x_data,
        gen_meth_theta_val,
        gen_meth_x,
        sep_fact,
        None,
        True,
        x_vals
    )
    y_to_sse = True
    val_sse_data = simulator.sim_data_to_sse_sim_data(
        method, val_data, exp_data, sep_fact, y_to_sse
    )
# Set validation data to None if not generating it
else:
    val_data = None
    val_sse_data = None
    gen_meth_theta_val = gen_meth_theta_val  # Value is None

# Define cs_name and cs_params class
cs_name = get_cs_class_from_val(cs_name_val).name #Save name of case study here
# Signac saves all BO_Results in different folders, so they can have the same name
cs_params = CaseStudyParameters(
    cs_name,
    ep0,
    sep_fact,
    normalize,
    kernel,
    lenscl,
    outputscl,
    retrain_GP,
    reoptimize_obj,
    gen_heat_map_data,
    bo_iter_tot,
    bo_runs_in_job,
    save_data,
    None,
    seed,
    ei_tol,
    obj_tol,
)
# Initialize driver class
driver = GPBO_Driver(
    cs_params,
    method,
    simulator,
    exp_data,
    sim_data,
    sim_sse_data,
    val_data,
    val_sse_data,
    None,
    ep_bias,
    gen_meth_theta,
)
# Get results
gpbo_res_simple, gpbo_res_GP = driver.run_bo_restarts()

res simp:  1
res simp:  2


In [4]:
gpbo_res_simple[0].results_df.iloc[:,0:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.852436438452487, -0.653516025469502]",4.589654e+00,11.348218,2.269644,"[2.0, -0.6466872466921301]",22.992176,14.244217,4.598435,2.848843,12.290787
1,2,5.411933,1.0,"[-1.7954176508166926, -1.1763903637519428]",0.000000e+00,79.201459,15.840292,"[0.99606086659948, -0.9971527775437947]",0.037933,0.000431,0.007587,0.000086,47.704426
2,3,5.411933,1.0,"[1.9856041217682576, -1.440173274833426]",2.527983e-30,16.301740,3.260348,"[1.44229547295586, -1.516487603374295]",10.961498,11.026074,2.192300,2.205215,46.306839
3,4,5.411933,1.0,"[2.0, -0.6213807480990085]",4.506545e+00,14.873986,2.974797,"[2.0, -0.6228366114455193]",24.041001,14.836576,4.808200,2.967315,19.084792


In [10]:
from bo_methods_lib.bo_methods_lib.analyze_data import General_Analysis, open_file_helper
results1 = open_file_helper("GPBO_test/workspace/8bab17d551702f04d63b2831d31f47e7/BO_Results2.gz")
results1[0].results_df.iloc[:,0:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.852349814712759, -1.0370190875972118]",2.446737e-03,7.311596,1.462319,"[0.9052129750651281, -1.047038160419906]",0.164331,0.165074,0.032866,0.033015,42.829080
1,2,5.411933,1.0,"[2.0, -2.0]",8.170034e+00,44.000000,8.800000,"[2.0, -2.0]",85.742432,44.000000,17.148486,8.800000,11.604968
2,3,5.411933,1.0,"[1.9856041217682576, -1.440173274833426]",9.500882e-321,16.301740,3.260348,"[1.0114478455257752, -0.9957777938170698]",0.000475,0.001917,0.000095,0.000383,30.807716
3,4,5.411933,1.0,"[2.0, -0.7225988313885122]",5.195090e+00,12.616348,2.523270,"[2.0, -0.7254302802195]",20.070907,12.563210,4.014181,2.512642,13.825024
4,5,5.411933,1.0,"[0.9954630109960431, -0.958794325809682]",5.357986e+00,0.057935,0.011587,"[0.9423816529482852, -0.9266754380268751]",0.211097,0.215999,0.042219,0.043200,34.617438


In [11]:
results2 = open_file_helper("GPBO_test/workspace/8bab17d551702f04d63b2831d31f47e7/BO_Results.gz")
results2[0].results_df.iloc[:,0:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.852349814712759, -1.0370190875972118]",2.446737e-03,7.311596,1.462319,"[0.9052129750651281, -1.047038160419906]",0.164331,0.165074,0.032866,0.033015,36.650572
1,2,5.411933,1.0,"[2.0, -2.0]",8.170034e+00,44.000000,8.800000,"[2.0, -2.0]",85.742432,44.000000,17.148486,8.800000,10.005512
2,3,5.411933,1.0,"[1.9856041217682576, -1.440173274833426]",9.500882e-321,16.301740,3.260348,"[1.0114478455257752, -0.9957777938170698]",0.000475,0.001917,0.000095,0.000383,26.708324
3,4,5.411933,1.0,"[2.0, -0.7225988313885122]",5.195090e+00,12.616348,2.523270,"[2.0, -0.7254302802195]",20.070907,12.563210,4.014181,2.512642,12.187082
4,5,5.411933,1.0,"[0.9954630109960431, -0.958794325809682]",5.357986e+00,0.057935,0.011587,"[0.9423816529482852, -0.9266754380268751]",0.211097,0.215999,0.042219,0.043200,30.360976


In [7]:
gpbo_res_simple[1].results_df.iloc[:,0:13]
#['Theta Min Obj', 'Theta Obj Act Cum', 'Min Obj Act', "Min Obj Act Cum"]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.792732962087873, 1.9394117601406493]",0.000000,300.049066,60.009813,"[1.1933723700482315, -1.2255720319671357]",2.082253,2.103942,0.416451,0.420788,27.030199
1,2,5.411933,1.0,"[0.9263579034964589, -1.9330159836032121]",8.122545,29.651872,5.930374,"[0.9263579034964589, -1.9330159836032121]",94.905152,29.651872,18.981030,5.930374,3.252149
2,3,5.411933,1.0,"[2.0, -0.6597805850057232]",4.194288,13.935475,2.787095,"[2.0, -0.6520655409575965]",24.637316,14.115985,4.927463,2.823197,20.214308


In [12]:
results1[1].results_df.iloc[:,0:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.792732962087873, 1.9394117601406493]",0.000000,300.049066,60.009813,"[0.9961876021333149, -0.9916397153994404]",0.000986,0.002522,0.000197,0.000504,25.739812
1,2,5.411933,1.0,"[0.9263579034964589, -1.9330159836032121]",8.122545,29.651872,5.930374,"[0.9263579034964589, -1.9330159836032121]",94.905152,29.651872,18.981030,5.930374,2.700825
2,3,5.411933,1.0,"[2.0, -0.6597804140507595]",4.194288,13.935478,2.787096,"[2.0, -0.6520653217041118]",24.637328,14.115990,4.927466,2.823198,15.492996


In [13]:
results2[1].results_df.iloc[:,0:13]

,BO Iter,Best Error,Exploration Bias,Theta Opt Acq,Opt Acq,Acq Obj Act,MSE Acq Act,Theta Min Obj,Min Obj GP,Min Obj Act,MSE Obj GP,MSE Obj Act,Time/Iter
0,1,5.411933,1.0,"[1.792732962087873, 1.9394117601406493]",0.000000,300.049066,60.009813,"[0.9961876021333149, -0.9916397153994404]",0.000986,0.002522,0.000197,0.000504,21.755822
1,2,5.411933,1.0,"[0.9263579034964589, -1.9330159836032121]",8.122545,29.651872,5.930374,"[0.9263579034964589, -1.9330159836032121]",94.905152,29.651872,18.981030,5.930374,2.328634
2,3,5.411933,1.0,"[2.0, -0.6597804140507595]",4.194288,13.935478,2.787096,"[2.0, -0.6520653217041118]",24.637328,14.115990,4.927466,2.823198,13.411884
